In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [2]:
# STEPS to get data

# get arr of notes
# get arr of keywords
# split note in arr
# for each word in keyword list, get count of it in arr (word_list.count(curr_keyword))
# also store which keywords are matched in arr
# have df (with cols patient id, encounter id, note TXT, keyword count, list of flagged keywords)
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])
    
# REPRESENTING data

# exported dataframe to csv with cols patient_id, num encounters keywords >= 1
# exported dataframe to csv that has general encounters where keywords >= 1, cols: patient id, encounter id, note txt, keyword count, matched keywords
# Summary Statistics: number of encounters with keywords >= 1, number of average encounters per patient with such encounters

In [262]:
#df1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_1.csv")
#df2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_2.csv")
#df3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_3.csv")
# df4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_4.csv")
# df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_5.csv")
df1_temp = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe33_notes_1_w_everything.csv")

In [263]:
#frames = [df1, df2]
notes = df1_temp

In [212]:
notes["NoteTXT"] = notes["NoteTXT"].str.replace('\s+', ' ', regex=True)
notes['NoteTXT'] = '-------------------------------------------------------------------------------------------------------------' + notes['NoteTXT'].astype(str)+'-------------------------------------------------------------------------------------------------------------'

In [213]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")

In [214]:
regex

,REGEX
0,dementia
1,cognition
2,memory
3,CDR
4,MMSE
5,MOCA
6,Alzheimer
7,cognitive\s*impairment
8,MCI
9,cerebellar


In [215]:
notes

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT
0,106692571,2871964,Z9527072,3.109615e+09,2016-04-04 00:00:00.0000000,Office Visit,960573028,Progress Notes,5,9.176024e+08,1.0,----------------------------------------------...
1,106692571,2871964,Z9527072,3.143458e+09,2017-02-08 00:00:00.0000000,Outpatient Surgery,1405329884,Progress Notes,5,1.346535e+09,1.0,----------------------------------------------...
2,106692571,2871964,Z9527072,3.147325e+09,2017-05-26 00:00:00.0000000,Office Visit,1529718266,Progress Notes,5,1.474581e+09,1.0,----------------------------------------------...
3,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1588808951,ED Triage Notes,5,1.535477e+09,1.0,----------------------------------------------...
4,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1589110426,ED Observation Initial Note,5,1.535821e+09,1.0,----------------------------------------------...
...,...,...,...,...,...,...,...,...,...,...,...,...
1035379,113727131,10127767464,Z16342458,3.219389e+09,2018-11-13 00:00:00.0000000,Office Visit,2215273251,Progress Notes,5,2.181061e+09,1.0,----------------------------------------------...
1035380,113727131,10127767464,Z16342458,3.253208e+09,2019-05-28 00:00:00.0000000,Orders Only,2476060048,Progress Notes,5,2.446971e+09,1.0,----------------------------------------------...
1035381,113727131,10127767464,Z16342458,3.365256e+09,2021-04-13 00:00:00.0000000,Office Visit,5637823913,Progress Notes,5,5.645510e+09,1.0,----------------------------------------------...
1035382,113727131,10127767464,Z16342458,3.366052e+09,2021-04-15 00:00:00.0000000,Telemedicine,5652882408,Progress Notes,5,5.649313e+09,1.0,----------------------------------------------...


In [216]:
locations = []
for r in tqdm(notes['NoteTXT']):
    try:
        locations.append([match.span() \
                  for match in re.finditer(re.compile("|".join(regex["REGEX"]), 
                                                  re.IGNORECASE),r)])
    except:
        locations.append([])

notes["regex_location"] = locations


100%|██████████████████████████████████████████████████████████████████████| 1035384/1035384 [14:34<00:00, 1184.55it/s]


In [143]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe33_notes_4_w_locations.csv")

In [217]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location
0,106692571,2871964,Z9527072,3.109615e+09,2016-04-04 00:00:00.0000000,Office Visit,960573028,Progress Notes,5,9.176024e+08,1.0,----------------------------------------------...,[]
1,106692571,2871964,Z9527072,3.143458e+09,2017-02-08 00:00:00.0000000,Outpatient Surgery,1405329884,Progress Notes,5,1.346535e+09,1.0,----------------------------------------------...,[]
2,106692571,2871964,Z9527072,3.147325e+09,2017-05-26 00:00:00.0000000,Office Visit,1529718266,Progress Notes,5,1.474581e+09,1.0,----------------------------------------------...,[]
3,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1588808951,ED Triage Notes,5,1.535477e+09,1.0,----------------------------------------------...,[]
4,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1589110426,ED Observation Initial Note,5,1.535821e+09,1.0,----------------------------------------------...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,106693931,10046146527,Z9535446,3.181732e+09,2017-12-08 00:00:00.0000000,Telephone,1782278640,Telephone Encounter,5,1.734144e+09,1.0,----------------------------------------------...,[]
96,106693931,10046146527,Z9535446,3.181732e+09,2017-12-08 00:00:00.0000000,Telephone,1782288323,Telephone Encounter,5,1.734155e+09,1.0,----------------------------------------------...,[]
97,106693931,10046146527,Z9535446,3.181738e+09,2017-12-08 00:00:00.0000000,Office Visit,1782585631,Progress Notes,5,1.734467e+09,1.0,----------------------------------------------...,[]
98,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Telephone,1783290666,Telephone Encounter,5,1.735196e+09,1.0,----------------------------------------------...,[]


In [218]:
l = []
for i in tqdm(range(len(notes["regex_location"]))):
    s = []
    try:
        for j in notes["regex_location"][i]:
            s.append(notes["NoteTXT"][i][j[0]-100 : j[0]+100])
    except:
        pass
    l.append(s)
    
    
notes["regex_sent"] = l

100%|█████████████████████████████████████████████████████████████████████| 1035384/1035384 [00:45<00:00, 22788.59it/s]


In [219]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_locations_sequences.csv")

In [220]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent
0,106692571,2871964,Z9527072,3.109615e+09,2016-04-04 00:00:00.0000000,Office Visit,960573028,Progress Notes,5,9.176024e+08,1.0,----------------------------------------------...,[],[]
1,106692571,2871964,Z9527072,3.143458e+09,2017-02-08 00:00:00.0000000,Outpatient Surgery,1405329884,Progress Notes,5,1.346535e+09,1.0,----------------------------------------------...,[],[]
2,106692571,2871964,Z9527072,3.147325e+09,2017-05-26 00:00:00.0000000,Office Visit,1529718266,Progress Notes,5,1.474581e+09,1.0,----------------------------------------------...,[],[]
3,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1588808951,ED Triage Notes,5,1.535477e+09,1.0,----------------------------------------------...,[],[]
4,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1589110426,ED Observation Initial Note,5,1.535821e+09,1.0,----------------------------------------------...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,106693931,10046146527,Z9535446,3.181732e+09,2017-12-08 00:00:00.0000000,Telephone,1782278640,Telephone Encounter,5,1.734144e+09,1.0,----------------------------------------------...,[],[]
96,106693931,10046146527,Z9535446,3.181732e+09,2017-12-08 00:00:00.0000000,Telephone,1782288323,Telephone Encounter,5,1.734155e+09,1.0,----------------------------------------------...,[],[]
97,106693931,10046146527,Z9535446,3.181738e+09,2017-12-08 00:00:00.0000000,Office Visit,1782585631,Progress Notes,5,1.734467e+09,1.0,----------------------------------------------...,[],[]
98,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Telephone,1783290666,Telephone Encounter,5,1.735196e+09,1.0,----------------------------------------------...,[],[]


In [221]:
notes['regex_sent'] = notes['regex_sent'].astype(str)
notes['regex_sent'] = notes['regex_sent'].str.replace('[', '')
notes['regex_sent'] = notes['regex_sent'].str.replace(']', '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [222]:
notes = notes[notes['regex_sent'].notna()]

In [223]:
notes['regex_sent'] = notes['regex_sent'].str.replace('"', "'")

In [224]:
notes = (notes.assign(regex_sent = notes['regex_sent'].str.split("', '"))
         .explode('regex_sent')
         .reset_index(drop=True))

In [225]:
l = []
for note in tqdm(notes["regex_sent"]):
    try:
        l.append(re.findall(re.compile("|".join(regex["REGEX"]), re.IGNORECASE), note))
    except:
        l.append([])

notes["regex_match"] = l

100%|███████████████████████████████████████████████████████████████████████| 278168/278168 [00:24<00:00, 11286.19it/s]


In [226]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_everything.csv")

In [227]:
notes

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match
0,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1593168404,Discharge Summary,5,1.539980e+09,1.0,----------------------------------------------...,"[(7211, 7220)]","'ma Neuro: Alert and oriented, no focal defici...",[Cognition]
1,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1494524615,Progress Notes,5,1.438352e+09,1.0,----------------------------------------------...,"[(2955, 2961)]","'apparent rashes, lesions or ulcers. Mental St...",[memory]
2,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1514654085,Letter,5,1.459068e+09,1.0,----------------------------------------------...,"[(3477, 3483)]","'apparent rashes, lesions or ulcers. Mental St...",[memory]
3,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Hospital Encounter,1783311709,ED Provider Notes,5,1.735375e+09,1.0,----------------------------------------------...,"[(8155, 8164)]",'sition Discharged: Home Condition on Discharg...,[cognition]
4,106693931,10046146527,Z9535446,3.294365e+09,2020-03-02 00:00:00.0000000,Office Visit,3773193065,Progress Notes,5,3.754601e+09,1.0,----------------------------------------------...,"[(7796, 7805)]",'rd/Spaulding PM&R Sports Medicine Newton-Well...,[cognition]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278163,113722742,10127721883,Z16337903,3.329813e+09,2020-11-04 00:00:00.0000000,Office Visit,4806472815,Progress Notes,5,4.806423e+09,1.0,----------------------------------------------...,"[(3680, 3689)]",'her lesions. Normal turgor. Psych: Alert and ...,[cognition]
278164,113722742,10127721883,Z16337903,3.335051e+09,2021-06-17 00:00:00.0000000,Office Visit,5986742692,Progress Notes,5,5.985886e+09,1.0,----------------------------------------------...,"[(12478, 12487)]","'eview, documentation, and care coordination. ...",[cognition]
278165,113722742,10127721883,Z16337903,3.358959e+09,2021-03-17 00:00:00.0000000,Telemedicine,5484338769,Progress Notes,5,5.485571e+09,1.0,----------------------------------------------...,"[(5455, 5464)]","'on and coordination, normal speech. Psych: Be...",[cognition]
278166,113722742,10127721883,Z16337903,3.362169e+09,2021-04-22 00:00:00.0000000,Office Visit,5695272756,Progress Notes,5,5.778178e+09,1.0,----------------------------------------------...,"[(7678, 7687)]","' Awake and oriented, normal speech. Psych: Be...",[cognition]


In [228]:
notes['regex_match'] = notes['regex_match'].astype(str)
notes['regex_match'] = notes['regex_match'].str.replace("\[\]", '')
notes = notes.replace(r'^\s*$', np.NaN, regex=True)

In [229]:
notes = notes[notes['regex_match'].notna()]

In [230]:
notes['regex_sent'] = notes['regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [231]:
import ast
notes.regex_match = notes.regex_match.apply(ast.literal_eval)

In [232]:
notes['regex_sent'] = notes['regex_sent'].apply(json.dumps)
notes['regex_match'] = notes['regex_match'].apply(json.dumps)

In [233]:
x = notes['regex_match'][97]
x = x.split(" ")
x

['["cognition"]']

In [234]:
print(len(x))

1


In [261]:
notes

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent
0,0,100000272,10040002098,Z6352398,3.048340e+09,1999-11-17 00:00:00.0000000,System Generated,330550893,Op Note,1,3.164291e+08,1.0,----------------------------------------------...,[],[]
1,1,100000272,10040002098,Z6352398,3.052785e+09,2009-03-15 00:00:00.0000000,System Generated,337968894,Discharge Summary,1,3.236400e+08,1.0,----------------------------------------------...,[],[]
2,2,100000272,10040002098,Z6352398,3.056972e+09,2001-02-19 00:00:00.0000000,System Generated,353890517,Discharge Summary,1,3.393630e+08,1.0,----------------------------------------------...,[],[]
3,3,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464327283,MR AVS Snapshot,1,4.466111e+08,1.0,----------------------------------------------...,[],[]
4,4,100000272,10040002098,Z6352398,3.076013e+09,2015-06-12 00:00:00.0000000,Office Visit,464415371,Progress Notes,1,4.466893e+08,1.0,----------------------------------------------...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079066,1079066,100365175,10059358563,Z8714148,3.375703e+09,2021-05-26 00:00:00.0000000,Patient Message,5878295268,Telephone Encounter,1,5.876515e+09,1.0,----------------------------------------------...,[],[]
1079067,1079067,100365175,10059358563,Z8714148,3.378998e+09,2021-06-14 00:00:00.0000000,Letter (Out),5966394734,Letter,1,5.965382e+09,1.0,----------------------------------------------...,[],[]
1079068,1079068,100365175,10059358563,Z8714148,3.379970e+09,2021-06-17 00:00:00.0000000,Patient Message,5994021567,Telephone Encounter,1,5.993225e+09,1.0,----------------------------------------------...,[],[]
1079069,1079069,100365175,10059358563,Z8714148,3.381402e+09,2021-06-23 00:00:00.0000000,Refill,6016856527,Telephone Encounter,1,6.016268e+09,1.0,----------------------------------------------...,[],[]


In [264]:
def regex_length(row):
    x = row['regex_match']
    x= x.split(" ")
    return len(x)

notes['match_count'] = notes.apply(regex_length, axis = 'columns')

In [265]:
notes.head()

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,453251959.0,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,492483305.0,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1
2,2,100000272,10040002098,Z6352398,3.082899e+09,2015-10-15 00:00:00.0000000,Office Visit,640846456,Progress Notes,1,612720888.0,1.0,----------------------------------------------...,"[(6049, 6057)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
3,3,100000272,10040002098,Z6352398,3.085967e+09,2015-08-20 00:00:00.0000000,Office Visit,563095698,Progress Notes,1,539570111.0,1.0,----------------------------------------------...,"[(3810, 3816)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1
4,4,100000272,10040002098,Z6352398,3.094749e+09,2015-10-14 00:00:00.0000000,Abstract,638983522,Progress Notes,1,610990078.0,1.0,----------------------------------------------...,"[(5554, 5562)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1


In [ ]:
#notes['match_count'] = notes.apply(regex_length, axis='columns')

In [236]:
notes.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_FINAL.csv")

In [237]:
notes.head(100)

,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1593168404,Discharge Summary,5,1.539980e+09,1.0,----------------------------------------------...,"[(7211, 7220)]","""Neuro: Alert and oriented, no focal deficits ...","[""Cognition""]",1
1,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1494524615,Progress Notes,5,1.438352e+09,1.0,----------------------------------------------...,"[(2955, 2961)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1
2,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1514654085,Letter,5,1.459068e+09,1.0,----------------------------------------------...,"[(3477, 3483)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1
3,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Hospital Encounter,1783311709,ED Provider Notes,5,1.735375e+09,1.0,----------------------------------------------...,"[(8155, 8164)]","""Discharged: Home Condition on Discharge: Sati...","[""cognition""]",1
4,106693931,10046146527,Z9535446,3.294365e+09,2020-03-02 00:00:00.0000000,Office Visit,3773193065,Progress Notes,5,3.754601e+09,1.0,----------------------------------------------...,"[(7796, 7805)]","""PM&R Sports Medicine Newton-Wellesley Spine *...","[""cognition""]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,106695664,4773623,Z9546313,3.186274e+09,2018-01-17 00:00:00.0000000,Hospital Encounter,1840320635,Discharge Summary,5,1.793885e+09,1.0,----------------------------------------------...,"[(19105, 19114)]","""bed comfortably in NAD, subxyphoid wound w/ s...","[""Cognition""]",1
96,106695664,4773623,Z9546313,3.193310e+09,2018-07-10 00:00:00.0000000,Office Visit,2050626671,Progress Notes,5,2.010640e+09,1.0,----------------------------------------------...,"[(7108, 7116)]","""are healthy - one son and one daughter brothe...","[""dementia""]",1
97,106695664,4773623,Z9546313,3.193310e+09,2019-03-19 00:00:00.0000000,Office Visit,2377501073,Progress Notes,5,2.350388e+09,1.0,----------------------------------------------...,"[(5642, 5651)]","""coordination and gait. Psychiatric: Alert and...","[""cognition""]",1
98,106695664,4773623,Z9546313,3.195631e+09,2018-04-02 00:00:00.0000000,Hospital Encounter,1927479164,Discharge Summary,5,1.883518e+09,1.0,----------------------------------------------...,"[(18676, 18685)]","""No edema. Feet are warm. Skin: No rash. Neuro...","[""Cognition""]",1


In [238]:
# to get # of general encouters --> do len(df[df["keyword_count"] >= 1])
# to get # of such encounters per patient
    # get list of unique patient ids
    # for each one, do len(df[df["patient_id"] == curr_patient_id]["encounter_id"])

In [266]:
encounters_with_more_than_1_keyword = notes[notes['match_count'] >= 1]
encounters_with_more_than_1_keyword

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1
2,2,100000272,10040002098,Z6352398,3.082899e+09,2015-10-15 00:00:00.0000000,Office Visit,640846456,Progress Notes,1,6.127209e+08,1.0,----------------------------------------------...,"[(6049, 6057)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
3,3,100000272,10040002098,Z6352398,3.085967e+09,2015-08-20 00:00:00.0000000,Office Visit,563095698,Progress Notes,1,5.395701e+08,1.0,----------------------------------------------...,"[(3810, 3816)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1
4,4,100000272,10040002098,Z6352398,3.094749e+09,2015-10-14 00:00:00.0000000,Abstract,638983522,Progress Notes,1,6.109901e+08,1.0,----------------------------------------------...,"[(5554, 5562)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270292,270292,100365175,10059358563,Z8714148,3.360340e+09,2021-04-23 00:00:00.0000000,Telemedicine,5693313879,Progress Notes,1,5.690061e+09,1.0,----------------------------------------------...,"[(964, 970)]",'he past 3 years. Interval events He was last ...,['memory'],1
270293,270293,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784497372,Progress Notes,1,5.781962e+09,1.0,----------------------------------------------...,"[(391, 400)]",' and a moderate mixed hearing loss rising to ...,['cognition'],1
270294,270294,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784981138,Progress Notes,1,5.782449e+09,1.0,----------------------------------------------...,"[(1194, 1203)]",'ge of Onset • Other Mother frequent falls • O...,['Alzheimer'],1
270295,270295,100365175,10059358563,Z8714148,3.366388e+09,2021-04-22 00:00:00.0000000,Follow-Up,5684835347,Progress Notes,1,5.681518e+09,1.0,----------------------------------------------...,"[(672, 678), (7858, 7867)]",'n recent months. He even thinks that on May 2...,['memory'],1


In [267]:
encounters_with_more_than_1_keyword.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches.csv")

In [241]:
unique_ids = notes["PatientID"].unique()
unique_ids = list(unique_ids)
len(unique_ids)

4200

In [242]:
summary_statistics_1 = pd.DataFrame(columns = ["patient_id", "encounters_with_keywords", "total_encounters"])
summary_statistics_1["patient_id"] = unique_ids
summary_statistics_1["encounters_with_keywords"] = [" "] * len(unique_ids)
summary_statistics_1["total_encounters"] = [" "] * len(unique_ids)
#summary_statistics_1["keywords"] = [" "] * len(unique_ids)

In [245]:
original_notes = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe33_5_locations_sequences.csv")

In [246]:
for i in tqdm(range(len(unique_ids))):
    summary_statistics_1["encounters_with_keywords"][i] = len(notes[notes["PatientID"] == unique_ids[i]])
    summary_statistics_1["total_encounters"][i] = len(original_notes[original_notes["PatientID"] == unique_ids[i]])

100%|██████████████████████████████████████████████████████████████████████████████| 4200/4200 [09:20<00:00,  7.49it/s]


In [247]:
summary_statistics_1

,patient_id,encounters_with_keywords,total_encounters
0,Z9527072,1,45
1,Z9535446,7,150
2,Z9536719,38,367
3,Z9542520,10,188
4,Z13197215,1,55
...,...,...,...
4195,Z16329469,48,173
4196,Z16331859,14,322
4197,Z16337611,7,237
4198,Z16337903,51,672


In [248]:
summary_statistics_1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\summary_statistics.csv")

In [294]:
x = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe_5_matches.csv")

In [295]:
x.head(25)

,Unnamed: 0,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count
0,0,106692571,2871964,Z9527072,3.163956e+09,2017-07-17 00:00:00.0000000,Hospital Encounter,1593168404,Discharge Summary,5,1.539980e+09,1.0,----------------------------------------------...,"[(7211, 7220)]","""Neuro: Alert and oriented, no focal deficits ...","[""Cognition""]",1
1,1,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1494524615,Progress Notes,5,1.438352e+09,1.0,----------------------------------------------...,"[(2955, 2961)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1
2,2,106693931,10046146527,Z9535446,3.152680e+09,2017-04-27 00:00:00.0000000,Office Visit,1514654085,Letter,5,1.459068e+09,1.0,----------------------------------------------...,"[(3477, 3483)]","""rashes, lesions or ulcers. Mental Status: Ale...","[""memory""]",1
3,3,106693931,10046146527,Z9535446,3.181874e+09,2017-12-09 00:00:00.0000000,Hospital Encounter,1783311709,ED Provider Notes,5,1.735375e+09,1.0,----------------------------------------------...,"[(8155, 8164)]","""Discharged: Home Condition on Discharge: Sati...","[""cognition""]",1
4,4,106693931,10046146527,Z9535446,3.294365e+09,2020-03-02 00:00:00.0000000,Office Visit,3773193065,Progress Notes,5,3.754601e+09,1.0,----------------------------------------------...,"[(7796, 7805)]","""PM&R Sports Medicine Newton-Wellesley Spine *...","[""cognition""]",1
5,5,106693931,10046146527,Z9535446,3.310090e+09,2020-06-22 00:00:00.0000000,Telemedicine,4151505288,Progress Notes,5,4.136271e+09,1.0,----------------------------------------------...,"[(6569, 6578)]","""PM&R Sports Medicine Newton-Wellesley Spine *...","[""cognition""]",1
6,6,106693931,10046146527,Z9535446,3.317045e+09,2020-11-09 00:00:00.0000000,Office Visit,4829960783,Progress Notes,5,4.820178e+09,1.0,----------------------------------------------...,"[(6902, 6908)]","""No edema. Feet are warm and distal pulses are...","[""Memory""]",1
7,7,106693931,10046146527,Z9535446,3.323949e+09,2020-10-21 00:00:00.0000000,Telemedicine,4704049082,Progress Notes,5,4.693240e+09,1.0,----------------------------------------------...,"[(2983, 2992)]","""Awake and oriented, normal speech Psych: Affe...","[""cognition""]",1
8,8,106694114,26662213,Z9536719,3.045401e+09,2013-03-14 00:00:00.0000000,System Generated,324227099,Op Note,5,3.103107e+08,1.0,----------------------------------------------...,"[(1641, 1644)]","""tissues in both flexion and extension. Attent...","[""mci""]",1
9,9,106694114,26662213,Z9536719,3.045637e+09,2012-03-02 00:00:00.0000000,System Generated,324535141,Op Note,5,3.106182e+08,1.0,----------------------------------------------...,"[(1577, 1580)]","""tissues in both flexion and extension. Attent...","[""mci""]",1


In [296]:
z = x[x['PatientID'] == 'Z9625126']
len(z)

293

In [268]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe_5_summary_statistics.csv")
s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe_4_summary_statistics.csv")
s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\3\apoe_3_summary_statistics.csv")
s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\2\apoe_2_summary_statistics.csv")
s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_summary_statistics.csv")

In [272]:
total_summary_stats = pd.concat([s1,s2,s3,s4,s5])

In [278]:
total_summary_stats.reset_index(inplace = True, drop=True)
total_summary_stats.drop(columns = ['Unnamed: 0'])

,patient_id,encounters_with_keywords,total_encounters
0,Z9527072,1,45
1,Z9535446,7,150
2,Z9536719,38,367
3,Z9542520,10,188
4,Z13197215,1,55
...,...,...,...
18041,Z8712370,18,172
18042,Z8713474,31,150
18043,Z8713489,22,222
18044,Z8713893,95,165


In [279]:
total_summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\total_summary_stats.csv")

In [290]:
avg_encounters_with_keywords = total_summary_stats["encounters_with_keywords"].sum() / len(total_summary_stats["encounters_with_keywords"])
avg_total_encounters = total_summary_stats["total_encounters"].sum() / len(total_summary_stats["total_encounters"])

print("Avg Encounters w/ Keywords Per Patient: ", avg_encounters_with_keywords, "\nAvg Total Encounters Per Patient: ", avg_total_encounters)
print("%, {}".format((avg_encounters_with_keywords/avg_total_encounters) * 100))

Avg Encounters w/ Keywords Per Patient:  72.88307658206804 
Avg Total Encounters Per Patient:  289.57325723151945
%, 25.169132425718654
